In [1]:
import numpy as np

In [2]:
class QAP_TabuSearch:
    def __init__(self, F, D, tenure=5, max_evals=1000):
        # F: Matriz de Flujo (Factories)
        # D: Matriz de Distancia (Locations)
        self.F = F
        self.D = D
        self.n = len(F)
        self.tenure = tenure
        self.max_evals = max_evals
        
        # Estado inicial: Una permutación aleatoria (NO randint)
        self.current_solution = np.random.permutation(self.n)
        self.best_solution = self.current_solution.copy()
        
        # Calculamos fitness inicial
        self.current_fitness = self.calculate_cost(self.current_solution)
        self.best_fitness = self.current_fitness
        
        # MEMORIA TABÚ: Matriz NxN. 
        # tabu_matrix[i][j] guardará el número de iteración hasta cuando es tabú intercambiar i y j
        self.tabu_matrix = np.zeros((self.n, self.n))

    def calculate_cost(self, solution):
        """
        Calcula el coste de forma VECTORIZADA (sin bucles for).
        Coste = sum(F * D_permutada)
        """
        # Reordenamos la matriz de distancias según la permutación actual
        D_permuted = self.D[solution][:, solution]
        # Multiplicamos elemento a elemento y sumamos
        return np.sum(self.F * D_permuted)

    def get_neighbors(self):
        """
        Genera todos los vecinos swap posibles y devuelve el mejor.
        Aplica Criterio de Aspiración y Lógica Tabú.
        """
        best_neighbor_sol = None
        best_neighbor_fit = float('inf') # Buscamos MINIMIZAR coste
        move_nodes = (-1, -1) # Guardamos qué nodos intercambiamos

        # Iteramos todos los pares posibles (i, j) para hacer swap
        # Esto es O(N^2), inevitable para explorar el vecindario completo
        for i in range(self.n):
            for j in range(i + 1, self.n):
                # 1. Crear vecino temporal
                neighbor = self.current_solution.copy()
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                
                # 2. Calcular fitness (optimizable calculando solo el delta, pero así es más claro)
                fit = self.calculate_cost(neighbor)
                
                # 3. VERIFICAR SI ES TABÚ O ASPIRACIÓN
                # ¿Es tabú este movimiento (i, j)?
                is_tabu = self.tabu_matrix[i][j] > 0
                
                # Criterio de Aspiración: Si es tabú pero es MEJOR que el mejor global, lo permitimos [cite: 3804]
                is_aspiration = fit < self.best_fitness
                
                if (not is_tabu) or (is_tabu and is_aspiration):
                    # Nos quedamos con el mejor vecino válido encontrado hasta ahora
                    if fit < best_neighbor_fit:
                        best_neighbor_fit = fit
                        best_neighbor_sol = neighbor
                        move_nodes = (i, j)

        return best_neighbor_sol, best_neighbor_fit, move_nodes

    def run(self):
        evals = 0
        iteration = 0
        
        # Bucle principal
        while evals < self.max_evals:
            iteration += 1
            
            # 1. Reducir contadores de la lista tabú (avanza el tiempo)
            # Restamos 1 a todo lo que sea > 0
            self.tabu_matrix[self.tabu_matrix > 0] -= 1
            
            # 2. Obtener el mejor vecino permitido (aunque sea peor que el actual)
            neighbor, fit, move = self.get_neighbors()
            
            if neighbor is None:
                break # No hay movimientos posibles (raro en swap)
            
            # 3. Movernos SIEMPRE a ese vecino
            self.current_solution = neighbor
            self.current_fitness = fit
            evals += 1 # Contamos evaluación (simplificado, realmente evaluamos N*N/2 por paso)
            
            # 4. Actualizar el mejor global si corresponde
            if fit < self.best_fitness:
                self.best_fitness = fit
                self.best_solution = neighbor.copy()
                print(f"Iter {iteration}: Nuevo record global! Coste: {self.best_fitness}")
            
            # 5. ACTUALIZAR LISTA TABÚ
            # Prohibimos deshacer el movimiento (i, j) durante 'tenure' turnos
            i, j = move
            self.tabu_matrix[i][j] = self.tenure
            self.tabu_matrix[j][i] = self.tenure # Simétrico
            
        return self.best_solution, self.best_fitness

In [3]:
# --- ZONA DE PRUEBAS (Generar datos sintéticos para validar) ---
N = 10
# Flujo aleatorio entre fábricas
F_test = np.random.randint(0, 10, size=(N, N))
# Distancia aleatoria entre ciudades (simétrica)
D_test = np.random.randint(0, 100, size=(N, N))
np.fill_diagonal(D_test, 0)

solver = QAP_TabuSearch(F_test, D_test, tenure=5, max_evals=500)
sol, cost = solver.run()
print(f"Solución final: {sol}")
print(f"Coste mínimo: {cost}")

Iter 1: Nuevo record global! Coste: 20278
Iter 2: Nuevo record global! Coste: 19878
Iter 3: Nuevo record global! Coste: 19303
Iter 4: Nuevo record global! Coste: 18795
Iter 5: Nuevo record global! Coste: 18727
Iter 9: Nuevo record global! Coste: 18708
Iter 10: Nuevo record global! Coste: 18572
Iter 11: Nuevo record global! Coste: 18392
Iter 13: Nuevo record global! Coste: 18375
Iter 17: Nuevo record global! Coste: 18337
Iter 18: Nuevo record global! Coste: 18275
Iter 25: Nuevo record global! Coste: 17716
Solución final: [4 6 2 1 3 9 7 8 0 5]
Coste mínimo: 17716
